In [1]:
#@Imports

#############################
# region: Imports and Setup #
#############################
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload
#############################
# endregion                 #
#############################

import datetime
from datetime import datetime as dt
from datetime import timedelta as td

from concordia.associative_memory import importance_function

from examples.custom_components.setup import model_setup, measurement_setup
from examples.tpb import utils
from examples.tpb import components as tpb_components
from examples.tpb import memory as tpb_memory
from examples.tpb.memory import TPBAgentConfig as AgentConfig

In [2]:
SETUP_TIME=dt(year=2024,month=10,day=1,hour=21)
TIME_STEP=td(minutes=5)

model, embedder = model_setup("llama3")
measurement, clock = measurement_setup(setup_time=SETUP_TIME,time_step=TIME_STEP)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [3]:
config = AgentConfig(
  name="Alice",
  gender="female",
  context="Alice loves to cook, but doesn't know what a Space Shuttle is.",
  goal="Alice wants to become a patissier.",
  traits="Determined, scatterbrained, picky",
  extras={
    'model': model,
    'embedder': embedder,  
    'importance': importance_model.importance,
    'clock_now': clock.now
    }
)

As a young girl, Alice was a whirlwind of energy and determination. She would spend hours in her mother's kitchen, experimenting with recipes and techniques, much to the chagrin of her scatterbrained nature, which often resulted in flour-covered countertops and pots and pans scattered across the floor. Despite this, Alice's love for cooking only grew stronger, and she dreamed of one day opening her own bakery. Her parents, though exasperated by her messiness, encouraged her passion, recognizing that it brought her such joy.

As Alice entered adulthood, her picky nature became more pronounced. She would spend hours deliberating over the perfect ingredients, the ideal cooking temperature, and the precise amount of seasoning to add. This attention to detail served her well in her career as a pastry chef, where she earned a reputation for creating exquisite desserts that were almost too beautiful to eat. Despite her success, Alice's scatterbrained tendencies continued to plague her, causin

Number of generated formative episodes (6) does not match number of formative ages (5).


In [4]:
behav = tpb_components.Behaviour(
  "behaviour",
  model=model,
  config=config,
  num_memories=100,
  num_behavs=5,
  clock_now=clock.now,
)

behav._verbose = True
behav._is_initialized = True
behav.update()

Based on Alice's personality and goals, here are 5 potential behaviors she could exhibit:

1. **Dive into research**: Alice could decide to learn more about Space Shuttles by reading books or watching documentaries, driven by her curiosity and determination.
2. **Seek guidance from a mentor**: Recognizing the importance of understanding the context of her goal (becoming a patissier), Alice might seek advice from an experienced pastry chef or culinary expert who can provide valuable insights and guidance.
3. **Create a plan for balance**: Acknowledging her tendency to get distracted, Alice could create a schedule or routine that balances her creative pursuits with more structured tasks, helping her stay focused and organized.
4. **Explore related topics**: Alice might decide to explore other areas of interest related to Space Shuttles, such as space exploration or the role of food in space travel, which could help her gain a deeper understanding of the topic and its relevance to her goa

In [5]:
attitude = tpb_components.Attitude(
  "attitude",
  model=model,
  config=config,
  components=[behav],
  num_memories=100,
  clock_now=clock.now,
  threading="DISABLED"
)

attitude._verbose = True
attitude._is_initialized = True
attitude.update()

Based on Alice's memories and traits, here are three potential positive and three potential negative consequences of her decision to "Dive into research" about Space Shuttles:

**Positive Consequences:**

1. **Improved understanding**: By learning more about Space Shuttles, Alice will gain a deeper understanding of the topic, which will enhance her knowledge and curiosity. (Value: 9, Likelihood: 80)
2. **New skills or perspectives**: As she delves into research, Alice may discover new skills or perspectives that can be applied to her passion for baking, such as attention to detail or creative problem-solving. (Value: 8, Likelihood: 60)
3. **Increased confidence**: The more Alice learns about Space Shuttles, the more confident she'll become in her ability to tackle complex topics and expand her knowledge. (Value: 7, Likelihood: 50)

**Negative Consequences:**

1. **Time-consuming distraction**: Researching Space Shuttles may divert Alice's attention away from her baking goals and passio

In [10]:
map(lambda x, y: x + y, ["a", "b", "c", "d", "e"], 5)

TypeError: 'int' object is not iterable